#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 08 - Numba and Cython - Exercise Solutions</center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

# Numba/Cython 

Recall the quicksort algorithm where we divide the unsorted list into sorted halves recursively:

In [3]:
import numpy as np


def qsort(arr):
    if len(arr) < 2:
        return arr
    else:
        pivot = arr[:1]
        rest = arr[1:]
        left = qsort(rest[rest <= pivot])
        right = qsort(rest[rest > pivot])

        return np.concatenate((left, pivot, right))


sortarr = qsort(np.random.rand(24))
print(sortarr)

[0.04738538 0.10212594 0.13503405 0.15141471 0.1860002  0.20493435
 0.2378761  0.25598357 0.31142673 0.3175635  0.38775255 0.39066275
 0.40743788 0.47707207 0.5403788  0.57338805 0.60088621 0.69719703
 0.71250038 0.71439115 0.71989031 0.74373576 0.80757828 0.82350922]


This takes some time for large arrays:

In [4]:
%timeit qsort(np.random.rand(100000).astype(np.float32))

321 ms ± 6.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can use Numba to speed things up when the arguments and operations are all known types like Numpy arrays:

In [5]:
from numba import jit


@jit(nopython=True)
def qsort_numba(arr):
    if len(arr) < 2:
        return arr
    else:
        pivot = arr[:1]
        rest = arr[1:]
        left = qsort_numba(rest[rest <= pivot])
        right = qsort_numba(rest[rest > pivot])

        return np.concatenate((left, pivot, right))


sortarr = qsort_numba(np.random.rand(24).astype(np.float32))
print(sortarr)

[0.05799419 0.08529736 0.11437497 0.11801493 0.15224953 0.18903235
 0.242708   0.2625493  0.3305468  0.33269766 0.34336498 0.3583178
 0.6261637  0.6574366  0.81262803 0.82482165 0.84997797 0.85754323
 0.875942   0.89214283 0.9035035  0.9043852  0.9501806  0.98601705]


In [6]:
%timeit qsort_numba(np.random.rand(100000).astype(np.float32))

60.6 ms ± 1.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Exercise 1: 

Implement an array re-ordering function:

In [7]:
@jit(nopython=True)
def reorder_array(arr, indices, out):
    """
    Define a function which fills in `out` with values from `arr` such that a value
    at position `i` is taken from `arr[indices[i]]`. Implement this in pure Python,
    see how slow it is, then use Numba to speed it up.
    """
    for i, ind in enumerate(indices):
        out[i] = arr[ind]


arr = np.random.rand(100000).astype(np.float32)
indices = np.arange(arr.shape[0])
out = np.zeros_like(arr)
np.random.shuffle(indices)

%timeit reorder_array(arr,indices,out)

95 µs ± 4.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Exercise 2:

Use Numba's in-built parallel abilities to speed up the function even more.

Reference: https://numba.pydata.org/numba-doc/dev/user/parallel.html#explicit-parallel-loops

You will want to use prange and parallel=True option for jit, nogil=True may be a good idea also

In [8]:
@jit(parallel=True, nogil=True)
def reorder_array_parallel(arr, indices, out):
    """
    Define a function which returns a new array of the same shape/type as `arr` with the
    value at position `i` is taken from `arr[indices[i]]`. Implement this in pure Python,
    see how slow it is, then use Numba to speed it up.
    """
    for i in prange(indices.shape[0]):
        out[i] = arr[indices[i]]


arr = np.random.rand(100000).astype(np.float32)
indices = np.arange(arr.shape[0])
out = np.zeros_like(arr)

np.random.shuffle(indices)

%timeit reorder_array_parallel(arr,indices,out)

<ipython-input-8-fa5481aefe30>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "reorder_array_parallel" failed type inference due to: Untyped global name 'prange': cannot determine Numba type of <class 'numba.core.ir.UndefinedType'>

File "<ipython-input-8-fa5481aefe30>", line 8:
def reorder_array_parallel(arr, indices, out):
    <source elided>
    """
    for i in prange(indices.shape[0]):
    ^

'prange' looks like a Numba internal function, has it been imported (i.e. 'from numba import prange')?

  @jit(parallel=True, nogil=True)
<ipython-input-8-fa5481aefe30>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "reorder_array_parallel" failed type inference due to: Untyped global name 'prange': cannot determine Numba type of <class 'numba.core.ir.UndefinedType'>

File "<ipython-input-8-fa5481aefe30>", line 8:
def reorder_array_parallel(arr, indices, out):
    <source elided>

NameError: global name 'prange' is not defined

### Exercise 3: 

Below is the code to draw a Mandelbrot fractal. This is a popular example of iterating over arrays. The code is terribly slow, speed it up by compiling both functions with Numba. Can parallelism be applied here?

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


@jit(nopython=True)
def mandelbrot(creal, cimag, max_iters):
    real = creal
    imag = cimag

    for n in range(max_iters):
        real2 = real * real
        imag2 = imag * imag
        
        if real2 + imag2 > 4.0:
            return n
        
        imag = 2 * real * imag + cimag
        real = real2 - imag2 + creal

    return 0


@jit(nopython=True, parallel=True)
def calc_mandelbrot(im, max_iters, centerx, centery, valuewidth, valueheight):
    w = im.shape[0]
    h = im.shape[1]
    
    for i in prange(w):
        for j in prange(h):
            creal = (i / w - 0.5) * valuewidth + centerx
            cimag = (j / h - 0.5) * valueheight + centery

            im[i, j] = mandelbrot(creal, cimag, max_iters)


max_iters = 100
im = np.zeros((1000, 1000))

%timeit calc_mandelbrot(im,max_iters,-0.75,0,3,3)

plt.figure(figsize=(20, 20))
plt.imshow(im * (im < max_iters), cmap="hot")

In [ ]:
max_iters = 2000
im = np.zeros((2000, 2000))

calc_mandelbrot(im, max_iters, -0.74875, 0.06507, 0.0001, 0.0001)

plt.figure(figsize=(20, 20))
plt.imshow(im * (im < max_iters), cmap="hot")

## Cython

We need to load the extension for Cython in the notebook:

In [ ]:
%load_ext cython

### Exercise 4: 

Implement the reordering in Cython:

In [ ]:
%%cython

cpdef void reorder_array_cython(float[:] arr,int[:] indices,float[:] out):
    """
    Define a function which fills in `out` with values from `arr` such that a value
    at position `i` is taken from `arr[indices[i]]`. Implement this in pure Python,
    see how slow it is, then use Numba to speed it up.
    """
    cdef int i, ind, n = arr.shape[0]
    for i in range(n):
        ind = indices[i]
        out[i] = arr[ind]

In [ ]:
arr = np.random.rand(100000).astype(np.float32)
indices = np.arange(arr.shape[0]).astype(np.int32)
out = np.zeros_like(arr)
np.random.shuffle(indices)

%timeit reorder_array_cython(arr,indices,out)

### Exercise 5: 

We can get more performance by using Cython parallel features. 
Re-implement the reorder function using Cython's prange. Compare speeds versus the above.

There are also decorators that may help: https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html#tuning-indexing-further

In [ ]:
%%cython 

import cython
from cython.parallel import prange

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cpdef void reorder_array_cython_parallel(float[:] arr,int[:] indices,float[:] out) nogil:
    """
    Define a function which fills in `out` with values from `arr` such that a value
    at position `i` is taken from `arr[indices[i]]`. Implement this in pure Python,
    see how slow it is, then use Numba to speed it up.
    """
    cdef int i, ind, n = arr.shape[0]
    for i in prange(n):
        ind = indices[i]
        out[i] = arr[ind]

In [ ]:
arr = np.random.rand(100000).astype(np.float32)
indices = np.arange(arr.shape[0]).astype(np.int32)
out = np.zeros_like(arr)
np.random.shuffle(indices)

%timeit reorder_array_cython_parallel(arr,indices,out)

### Exercise 6: re-implement the Mandelbrot program with Cython. 

I have provided the function prototypes again, you should be using `cdef` to declare variables and `prange` where appropriate.

In [ ]:
%%cython 

import cython
from cython.parallel import prange

cdef int mandelbrot_cython(double creal, double cimag, int max_iter) nogil:
    cdef:
        double real2, imag2
        double real = creal, imag = cimag
        int n

    for n in range(max_iter):
        real2 = real*real
        imag2 = imag*imag
        if real2 + imag2 > 4.0:
            return n
        imag = 2* real*imag + cimag
        real = real2 - imag2 + creal
        
    return 0

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cpdef void calc_mandelbrot_cython(double[:,:] im,int max_iters,double centerx,double centery,double valuewidth,double valueheight) nogil:
    cdef:
        int w,h,i,j
        float creal,cimag
        
    w=im.shape[0]
    h=im.shape[1]
    
    for i in prange(w):
        for j in prange(h):
            creal = (i/w-0.5)*valuewidth+centerx
            cimag = (j/h-0.5)*valueheight+centery
            im[i,j]=mandelbrot_cython(creal,cimag,max_iters)

In [ ]:
max_iters=100
im = np.zeros((1000, 1000))

%timeit calc_mandelbrot_cython(im,max_iters,-0.75,0,3,3)

plt.figure(figsize=(20, 20))
plt.imshow(im * (im < max_iters), cmap="hot")

## Bonus: 

Mandelbrot implemented using `numba.vectorize`. There's nothing to do here, it's just an interesting implementation which is fastest of those above.

In [ ]:
from numba import vectorize


@vectorize(["float64(float64, float64, int32)"], target="parallel")
def mandelbrot_v(creal, cimag, max_iters):
    real = creal
    imag = cimag

    for n in range(max_iters):
        real2 = real * real
        imag2 = imag * imag

        if real2 + imag2 > 4.0:
            return n

        imag = 2 * real * imag + cimag
        real = real2 - imag2 + creal

    return 0


def mandelbrot_vectorize(minx, maxx, miny, maxy, width, height, max_iters):
    x, y = np.mgrid[minx : maxx : (maxx - minx) / width, miny : maxy : (maxy - miny) / height]

    return mandelbrot_v(x, y, max_iters)

In [ ]:
max_iters=100
%timeit im=mandelbrot_vectorize(-2.,1,-1.5,1.5,1000,1000,max_iters)

plt.figure(figsize=(20, 20))
plt.imshow(im * (im < max_iters), cmap="afmhot")